In [13]:
import pandas as pd
import  numpy as np
from category_encoders import *
import  category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [14]:
#reading the data and dropping NAs
dataset = pd.read_csv("~/Desktop/DataDump/Flights_TRAIN.csv")
dataset = dataset.dropna()
#In case I want to take a subsample of the data
dataset = dataset.sample(frac=1, random_state=1234).reset_index(drop=True)

In [15]:
#Defining cols that require encoding and cols to go to X
cols_to_encode = ["origin", "dest","op_unique_carrier"]
cols_X = ['dep_dayofweek','op_unique_carrier', 'origin', 'dest',
       'arr_Fog', 'arr_Snow', 'arr_Cold', 'arr_Storm', 'arr_Hail',
          'arr_Precipitation', 'crs_dep_time', 'crs_arr_time',
          'flights_origin', 'flights_dest']

In [16]:
#defining X and y
X = dataset[cols_X]
y = dataset["arr_delay"]

In [17]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1234)

In [18]:
#Target encoding for categorical variable
encoder = TargetEncoder(cols=cols_to_encode, smoothing=1)
encoder.fit(X_train, y_train)
X_train_fitted = encoder.transform(X_train)
X_test_fitted = encoder.transform(X_test)

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=5)
pca.fit(X_train)
print(pca.explained_variance_ratio_)

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Lasson regression

In [21]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

In [16]:
params = {
    'alpha': [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1]
}

In [29]:
#Standardization
std = StandardScaler()
X_train_reg = std.fit_transform(X_train_fitted)
X_test_reg = std.transform(X_test_fitted)

In [36]:
#Lasso Regression
reg = Lasso()
#Random search
random_search = RandomizedSearchCV(reg, param_distributions=params, n_iter=5, n_jobs=-1, verbose=10, random_state=1234)
random_search.fit(X_train_reg, y_train)
best_reg = random_search.best_estimator_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [33]:
print(f"R2: {best_reg.score(X_test_reg,y_test)}")
y_pred = best_reg.predict(X_test_reg)
print(f"MSE: {metrics.mean_squared_error(y_test,y_pred)}")

R2: 0.04571405092666547
MSE: 2067.4968618532707


### Random Forest

In [11]:
rfr = RandomForestRegressor()

In [8]:
params = {
    'max_depth': [10, 20, 50, None],
    'n_estimators': [50, 100, 200]

}

In [12]:
random_search = RandomizedSearchCV(rfr, param_distributions=params, n_iter=5, n_jobs=-1, verbose=5, random_state=1234, cv = 4)
random_search.fit(X_train_fitted, y_train)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


RandomizedSearchCV(cv=4, estimator=RandomForestRegressor(), n_iter=5, n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 50, None],
                                        'n_estimators': [50, 100, 200]},
                   random_state=1234, verbose=5)

In [13]:
best_rfr = random_search.best_estimator_

In [14]:
print(f"R2: {best_rfr.score(X_test_fitted,y_test)}")
y_pred = best_rfr.predict(X_test_fitted)
print(f"MSE: {metrics.mean_squared_error(y_test,y_pred)}")
#best so far

R2: 0.08231807509936029
MSE: 1988.1928490657663


### SVM

In [19]:
from sklearn.svm import SVR

In [23]:
svr_poly = SVR()

In [24]:
params = {
    'kernel': ["poly","rbf"],
    'C': [0.1,1,10],
    'degree': [1, 2, 3, 4, 5]
}

In [ ]:
random_search = RandomizedSearchCV(svr_poly, param_distributions=params, n_iter=4, n_jobs=-1, verbose=5, random_state=1234, cv = 4)
random_search.fit(X_train_fitted, y_train)

Fitting 4 folds for each of 4 candidates, totalling 16 fits


In [ ]:
best_SVM = random_search.best_estimator_

In [ ]:
print(f"R2: {best_SVM.score(X_test_fitted,y_test)}")
y_pred = best_SVM.predict(X_test_fitted)
print(f"MSE: {metrics.mean_squared_error(y_test,y_pred)}")

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.